# Tutorial [Raytracing em um final de semana](https://raytracing.github.io/books/RayTracingInOneWeekend.html#rays,asimplecamera,andbackground)

Observações: 

- As **etapas 1 e 2** estão implementadas no diretório **Atividade 01**

- A **etapa 3** está implementada em **Atividade 02**

- A **etapa 4.1** está implementada em **Ray.py** no mesmo repositório

## Importações

In [1]:
import sys
import os

import numpy as np
from tqdm import tqdm

sys.path.append(os.path.abspath(os.path.join('..')))

from Ray import Ray
from Atividade02.src.vectorized.Vec3 import Vec3, Point3, Color
from Atividade01.src.ImageIO import ImageWriter
from Atividade01.src.Image import Image

## 4.2 Sending Rays Into the scene 

In [2]:
aspect_ratio = 16.0 / 9.0
image_width = 400

image_height = int(image_width / aspect_ratio)
if image_height < 1:
    image_height = 1

viewport_height = 2.0
viewport_width = viewport_height * (image_width / image_height)
# OBS: viewport_width: não está usando a variável aspect_ratio, pois a variável aspect_ratio é a proporção ideal da imagem, porém, arredondamos image_height para inteiro, podendo assim perder um pouco de precisão na proporção da imagem.

# PAREI NA CAMERA...

In [3]:
def ray_color(ray: Ray):
    unit_direction = ray.direction.unit_vector()
    a = 0.5 * (unit_direction.y + 1.0)
    return (1.0 - a) * Color([1.0, 1.0, 1.0]) + a * Color([0.5, 0.7, 1.0])

focal_length = 1.0
camera_center = Point3([0, 0, 0])

viewport_u = Vec3([viewport_width, 0, 0])
viewport_v = Vec3([0, -viewport_height, 0])

pixel_delta_u = viewport_u / image_width
pixel_delta_v = viewport_v / image_height

viewport_upper_left = camera_center - Vec3([0, 0, focal_length]) - (viewport_u / 2) + (viewport_v / 2)
pixel00_loc = viewport_upper_left + 0.5 * (pixel_delta_u + pixel_delta_v)

# CRIAR MATRIZ
image = Image(image_width, image_height)
for j in tqdm(range(image_height)):
    for i in range(image_width):
        pixel_center = pixel00_loc + i * pixel_delta_u - j * pixel_delta_v
        ray_direction = pixel_center - camera_center
        ray = Ray(camera_center, ray_direction)

        pixel_color = ray_color(ray)
        image[j, i] = pixel_color

img_writer = ImageWriter(image)
img_writer.save('teste.png')

  0%|          | 0/225 [00:00<?, ?it/s]

100%|██████████| 225/225 [00:05<00:00, 38.78it/s]
